# ACA-Py & ACC-Py Basic Template

## Copy this template into the root folder of your notebook workspace to get started

### Imports

In [14]:
from aries_cloudcontroller import AriesAgentController
import os
import time
from termcolor import colored

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://alice-agent:3021 and an api key of alicesSecretApiKey


### Start a Webhook Server

In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Register Agent Event Listeners

You can see some examples within the webhook_listeners recipe. Copy any relevant cells across and customise as needed.

In [5]:
listeners = []

# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))



connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)

def issuer_handler(payload):
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    print("\n---------------------------------------------------\n")
    print("Handle Issue Credential Webhook")
    print(f"Connection ID : {connection_id}")
    print(f"Credential exchange ID : {exchange_id}")
    print("Agent Protocol Role : ", role)
    print("Protocol State : ", state )
    print("\n---------------------------------------------------\n")
    
    
    if state == "offer_sent":
        proposal = payload["credential_proposal_dict"]
        attributes = proposal['credential_proposal']['attributes']
        print(f"Offering : \n {attributes}")
        ## YOUR LOGIC HERE
    elif state == "request_received":
        print("Request for credential received")
        ## YOUR LOGIC HERE
    elif state == "credential_sent":
        print("Credential Sent")
        ## YOUR LOGIC HERE
    
issuer_listener = {
    "topic": "issue_credential",
    "handler": issuer_handler
}

listeners.append(issuer_listener)


def verifier_proof_handler(payload):
    role = payload["role"]
    connection_id = payload["connection_id"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print("\n---------------------------------------------------------------------\n")
    print("Handle present-proof")
    print("Connection ID : ", connection_id)
    print("Presentation Exchange ID : ", pres_ex_id)
    print("Protocol State : ", state)
    print("Agent Role : ", role)
    print("Initiator : ", payload["initiator"])
    print("\n---------------------------------------------------------------------\n")
    

    if state == "request_sent":
        print("Presentation Request\n")
        print(payload["presentation_request"])
        print("\nThe presentation request is encoded in base64 and packaged into a DIDComm Message\n")
        print(payload["presentation_request_dict"])
        print("\nNote the type defines the protocol present-proof and the message request-presentation\n")
        print(payload["presentation_request_dict"]["@type"])
    elif state == "presentation_received":
        print("Presentation Received")
    else:
        print(f"Presentation Verified? {payload['verified']} \n")
        
verifier_listener = {
    "topic": "present_proof",
    "handler": verifier_proof_handler
}

listeners.append(verifier_listener)

agent_controller.register_listeners(listeners)

## Store Schema and Cred Def ID's

In [6]:
schema_id="J1y6c8mp32uBxVHkRiLM71:2:aca-acc-jupyter-plyaground:0.0.1"
cred_def_id="J1y6c8mp32uBxVHkRiLM71:3:CL:216776:default"

## Accept Invitation

Copy an invitation object from another agent playing the role inviter (see the inviter_template recipe)

In [8]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'e743b38b-fab7-4315-92a7-1b7b6cab0f95', 'recipientKeys': ['36CZQj9EpiGbPG926PU7GrnApdqJgpGE91dKQHWHEzZS'], 'serviceEndpoint': 'https://588d5381c2e8.ngrok.io', 'label': 'Bob'}


In [9]:
auto_accept="false"
alias=None

invite_response = await agent_controller.connections.receive_invitation(invitation, alias, auto_accept)
connection_id = invite_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  48a27664-2266-4382-9001-ced139060721
State :  invitation
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
invitation


In [10]:
# Label for the connection
my_label = None
# Endpoint you expect to recieve messages at
my_endpoint = None

accept_response = await agent_controller.connections.accept_invitation(connection_id, my_label, my_endpoint)

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  48a27664-2266-4382-9001-ced139060721
State :  request
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
request
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  48a27664-2266-4382-9001-ced139060721
State :  response
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
response
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  48a27664-2266-4382-9001-ced139060721
State :  active
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
Connection ID: 48a27664-2266-4382-9001-ced139060721 is now active.


## Populate Credential Attributes

Before you can issue a credential, you must define the values that will be issued in this credential. The attribute names **MUST** match those in the schem identified by the <schema_id> value.

Make sure to change all code enclosed with <>.


In [11]:
example=input("Whatever you want. This can be any string. Including a serialized JSON, or a base64 encoded image: ")
credential_attributes = [
    {"name": "example", "value": example},
]
print(credential_attributes)

Whatever you want. This can be any string. Including a serialized JSON, or a base64 encoded image:  sOME DATA


[{'name': 'example', 'value': 'sOME DATA'}]


## Send Credential

This is the easiest way to issue a credential because it automates the rest of the protocol steps. 

Note: The `connection_id` must be in the active state before a credential can be sent.

In [12]:
# Do you want the ACA-Py instance to trace it's processes (for testing/timing analysis)
trace = False
comment = ""
# Remove credential record after issued?
auto_remove = True

# Change <schema_id> and <cred_def_id> to correct pair. Cred_def_id must identify a definition to which your agent has corresponding private issuing key.
send_cred_response = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, comment, auto_remove, trace)

# Note last three args are optional.
# await agent_controller.issuer.send_credential(connection_id, <schema_id>, <cred_def_id, credential_attributes)


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 48a27664-2266-4382-9001-ced139060721
Credential exchange ID : 05efd17c-dab7-4ae4-b232-9a392db07f7f
Agent Protocol Role :  issuer
Protocol State :  offer_sent

---------------------------------------------------

Offering : 
 [{'name': 'example', 'value': 'sOME DATA'}]

---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 48a27664-2266-4382-9001-ced139060721
Credential exchange ID : 05efd17c-dab7-4ae4-b232-9a392db07f7f
Agent Protocol Role :  issuer
Protocol State :  request_received

---------------------------------------------------

Request for credential received

---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 48a27664-2266-4382-9001-ced139060721
Credential exchange ID : 05efd17c-dab7-4ae4-b232-9a392db07f7f
Agent Protocol Role :  issuer
Protocol State :  credential_issued

---------

## Now Request Proof of Credential Issued


In [23]:
# We add a constraint that the attribute must originate from this schema

# trusted_issuer_did = "<SOME ISSUER DID ON INDY NETWORK>"

# cred_def_id = "<SOME CRED DEF>"

# Define the list of attributes and restrictions under which each attribute was issued that a prover must satisfy with a presentation
# NOTE: if identifying a schema or credential definition then the attribute name must be contained within the corresponding schema.
req_attrs = [
    {"name": "example", "restrictions": [{"schema_id": schema_id, "cred_def_id": cred_def_id}]},
]

# We could extend this to request the name attribute aswell if we wanted.


indy_proof_request = {
    "name": "Proof of Data Owner",
    "version": "1.0",
    "requested_attributes": {
        # They generally follow this uuid pattern. Unique identifier for attribute within context of this proof request
        # Note that req_attr['name'] gets the attribute name of each object. E.g. domain and name in this case
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    # Predicates allow us to specify range proofs or set membership on attributes. For example greater than 10.
    # We will ignore these for now.
    "requested_predicates": {
#         f"0_{req_pred['name']}_GE_uuid":
#         req_pred for req_pred in req_preds
    },
    # You can also request the entire proof request be non-revoked
#     "non_revoked":  {"to": int(time.time())}
}

In [24]:
proof_request = {
    "comment": "Some optional comment",
    "connection_id": connection_id,
    "proof_request": indy_proof_request,
    # Do you want your agent to trace this request (for debugging)
    "trace": False
}

proof_request_response = await agent_controller.proofs.send_request(proof_request)


---------------------------------------------------------------------

Handle present-proof
Connection ID :  48a27664-2266-4382-9001-ced139060721
Presentation Exchange ID :  edc84101-5d86-4e8a-a0a1-6a7cb2127675
Protocol State :  request_sent
Agent Role :  verifier
Initiator :  self

---------------------------------------------------------------------

Presentation Request

{'name': 'Proof of Data Owner', 'version': '1.0', 'requested_attributes': {'0_example_uuid': {'name': 'example', 'restrictions': [{'schema_id': 'J1y6c8mp32uBxVHkRiLM71:2:aca-acc-jupyter-plyaground:0.0.1', 'cred_def_id': 'J1y6c8mp32uBxVHkRiLM71:3:CL:216776:default'}]}}, 'requested_predicates': {}, 'nonce': '43448558002334650989233'}

The presentation request is encoded in base64 and packaged into a DIDComm Message

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': 'bb9dc7d4-4a59-4e5f-80c7-fca8eac74eba', 'request_presentations~attach': [{'@id': 'libindy-request-presentatio

## Get Presentation Exchange Record

This record keeps track of the current state of the presentation protocol, which must be in the `presentation_received` state before the presentation can be verified.

Note: This could also happen in the webhook logic.

In [26]:
presentation_exchange_id = proof_request_response["presentation_exchange_id"]

## Verify Presentation

Only if it is in the right state. 

Note: Verifying a presentation moves the state to `verified` regardless of whether the presentation request has been satisfied. To check this you must refer to the `verified` property on the response.

In [27]:
verified_response = await agent_controller.proofs.verify_presentation(presentation_exchange_id)

verified = verified_response["verified"]


---------------------------------------------------------------------

Handle present-proof
Connection ID :  48a27664-2266-4382-9001-ced139060721
Presentation Exchange ID :  edc84101-5d86-4e8a-a0a1-6a7cb2127675
Protocol State :  verified
Agent Role :  verifier
Initiator :  self

---------------------------------------------------------------------

Presentation Verified? true 



## Parsing Disclosed Attribute Values from Presentation

A presentation object contains three classes of attributes. 
* Revealed Attributes: Attributes that were signed by an issuer and have been revealed in the presentation process
* Self Attested Attributes: Attributes that the prover has self attested to in the presentation object.
* Predicate proofs: Attribute values that have been proven to meet some statement. (TODO: Show how you can parse this information)

### Parse Revealed Attributes

In [32]:
for (name, val) in verified_response['presentation']['requested_proof']['revealed_attrs'].items():
    ## This is the actual data that you want. It's a little hidden
    print("\nAttribute : ", val)
    
    attr_name = verified_response["presentation_request"]["requested_attributes"][name]["name"]
    print("Attribute Name :  Raw Value")
    print(f"{attr_name}   :  {val['raw']}")


Attribute :  {'sub_proof_index': 0, 'raw': 'sOME DATA', 'encoded': '54601064853684362018060759795266104659214728646083330886433293175925246179662'}
Attribute Name :  Raw Value
example   :  sOME DATA


## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()